In [ ]:
using Pkg
Pkg.activate(".")
Pkg.resolve()
Pkg.instantiate()

using CSV
using DataFrames
using Chain
using StatsBase # for competerank
using ShiftedArrays: lag # for lag()
using Dates

  Activating project at `c:\Users\ouhsj\Julia_Practice\Julia_Practice`
  No Changes to `C:\Users\ouhsj\Julia_Practice\Julia_Practice\Project.toml`
  No Changes to `C:\Users\ouhsj\Julia_Practice\Julia_Practice\Manifest.toml`


In [9]:
df_customer = CSV.read("data/customer.csv", DataFrame)
df_category = CSV.read("data/category.csv", DataFrame)
df_product = CSV.read("data/product.csv", DataFrame)
df_receipt = CSV.read("data/receipt.csv", DataFrame)
df_store = CSV.read("data/store.csv", DataFrame)
df_geocode = CSV.read("data/geocode.csv", DataFrame);

In [10]:
# p1
first(df_receipt, 10)

Row,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
,Int64,Int64,String7,Int64,Int64,String15,String15,Int64,Int64
1,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
2,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
3,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
4,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
5,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
6,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
7,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
8,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
9,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770


In [11]:
# p2

# 解答1
first(df_receipt[!, [:sales_ymd, :customer_id, :product_cd, :amount]], 10)

# 解答2
@chain df_receipt[!, [:sales_ymd, :customer_id, :product_cd, :amount]] begin 
    first(10)
end

# 解答3
@chain df_receipt begin
    select(:sales_ymd, :customer_id, :product_cd, :amount)
    first(10)
end

Row,sales_ymd,customer_id,product_cd,amount
,Int64,String15,String15,Int64
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


In [12]:
# p3
rename!(df_receipt, :sales_ymd => :sales_date)
first(df_receipt[!,[:sales_date, :customer_id, :product_cd, :amount]], 10)

@chain df_receipt begin
    select(:sales_date, :customer_id, :product_cd, :amount)
    first(10)
end

Row,sales_date,customer_id,product_cd,amount
,Int64,String15,String15,Int64
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


In [13]:
# p4
rename!(df_receipt, :sales_date => :sales_ymd)

# 解答1
df_receipt[!, [:sales_ymd, :customer_id, :product_cd, :amount]] |>
    filter(row -> row.customer_id == "CS018205000001")

# 解答2
@chain df_receipt begin
        select(:sales_ymd, :customer_id, :product_cd, :amount)
        filter(row -> row.customer_id == "CS018205000001", _) # chainでfilterする場合は，dataの場所に_を使用する．
    end

Row,sales_ymd,customer_id,product_cd,amount
,Int64,String15,String15,Int64
1,20180911,CS018205000001,P071401012,2200
2,20180414,CS018205000001,P060104007,600
3,20170614,CS018205000001,P050206001,990
4,20170614,CS018205000001,P060702015,108
5,20190216,CS018205000001,P071005024,102
6,20180414,CS018205000001,P071101002,278
7,20190226,CS018205000001,P070902035,168
8,20190924,CS018205000001,P060805001,495
9,20190226,CS018205000001,P071401020,2200


In [14]:
# p7
df_receipt[!, [:sales_ymd, :customer_id, :product_cd, :amount]] |>
    filter(row -> row.customer_id == "CS018205000001") |>
    filter(row -> row.amount >= 1000)

Row,sales_ymd,customer_id,product_cd,amount
,Int64,String15,String15,Int64
1,20180911,CS018205000001,P071401012,2200
2,20190226,CS018205000001,P071401020,2200
3,20180911,CS018205000001,P071401005,1100


In [15]:
# p8
df_receipt[!, [:sales_ymd, :customer_id, :product_cd, :amount]] |>
    filter(row -> row.customer_id == "CS018205000001") |>
    filter(row -> row.product_cd != "P071401019")

Row,sales_ymd,customer_id,product_cd,amount
,Int64,String15,String15,Int64
1,20180911,CS018205000001,P071401012,2200
2,20180414,CS018205000001,P060104007,600
3,20170614,CS018205000001,P050206001,990
4,20170614,CS018205000001,P060702015,108
5,20190216,CS018205000001,P071005024,102
6,20180414,CS018205000001,P071101002,278
7,20190226,CS018205000001,P070902035,168
8,20190924,CS018205000001,P060805001,495
9,20190226,CS018205000001,P071401020,2200


In [16]:
# p9
df_store |>
    filter(row -> row.prefecture_cd != "13" && row.floor_area <= 900)

Row,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String7,String31,Int64,String15,String,String,String15,Float64,Float64,Float64
1,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831.0
2,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.632,35.5466,890.0
3,S13008,成城店,13,東京都,東京都世田谷区成城三丁目,トウキョウトセタガヤクセイジョウサンチョウメ,03-0123-4012,139.597,35.6361,883.0
4,S13020,十条仲原店,13,東京都,東京都北区十条仲原三丁目,トウキョウトキタクジュウジョウナカハラサンチョウメ,03-0123-4019,139.719,35.7669,801.0
5,S13043,南品川店,13,東京都,東京都品川区南品川三丁目,トウキョウトシナガワクミナミシナガワサンチョウメ,03-0123-4027,139.744,35.6098,845.0
6,S13041,八王子店,13,東京都,東京都八王子市大塚,トウキョウトハチオウジシオオツカ,042-123-4026,139.423,35.6379,810.0
7,S13017,高円寺南店,13,東京都,東京都杉並区高円寺南四丁目,トウキョウトスギナミクコウエンジミナミヨンチョウメ,03-0123-4016,139.651,35.7027,841.0
8,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.022,35.6612,808.0
9,S13037,南砂一丁目店,13,東京都,東京都江東区南砂一丁目,トウキョウトコウトウクミナミスナイッチョウメ,03-0123-4023,139.821,35.679,814.0


In [17]:
# p10
@chain df_store |>
    filter(row -> startswith(row.store_cd, "S14")) begin
        first(10)
    end

Row,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String7,String31,Int64,String15,String,String,String15,Float64,Float64,Float64
1,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732.0
2,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495.0
3,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.372,35.5733,1679.0
4,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.524,1548.0
5,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.496,35.4592,1830.0
6,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.496,35.463,1574.0
7,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.658,35.4216,1341.0
8,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831.0
9,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.579,35.2964,1838.0


In [18]:
# p11
@chain df_customer |>
    filter(row -> endswith(row.customer_id, "1")) begin
        first(10)
    end

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
3,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
4,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
5,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
6,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
7,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
8,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
9,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0


In [19]:
# p12
df_store |>
    filter(row -> contains(row.address, "横浜市"))

Row,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String7,String31,Int64,String15,String,String,String15,Float64,Float64,Float64
1,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732.0
2,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495.0
3,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.524,1548.0
4,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.496,35.4592,1830.0
5,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.496,35.463,1574.0
6,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.658,35.4216,1341.0
7,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831.0
8,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.632,35.5466,890.0
9,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.576,35.5491,1657.0


In [20]:
# p13

# 解答1
@chain df_customer begin
    filter(row -> startswith(row.status_cd, ('A', 'B', 'C', 'D', 'E', 'F')), _) # chainでfilterする場合は，_を使用する．
    first(10)
end

# 解答2
@chain df_customer |>
    filter(row -> startswith(row.status_cd, "A") ||
        startswith(row.status_cd, "B") ||
        startswith(row.status_cd, "C") ||
        startswith(row.status_cd, "D") ||
        startswith(row.status_cd, "E") ||
        startswith(row.status_cd, "F")) begin
            first(10)
        end

# 解答3
first(filter(row -> startswith(row.status_cd, ('A', 'B', 'C', 'D', 'E', 'F')), df_customer), 10)

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15
1,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
2,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
3,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
4,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
5,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
6,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
7,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
8,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
9,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E


In [21]:
# p14
@chain df_customer begin
    filter(row -> endswith(row.status_cd, ('1', '2', '3', '4', '5', '6', '7', '8', '9')), _) 
    first(10)
end

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15
1,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
2,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
3,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
4,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
5,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
6,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
7,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
8,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
9,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1


In [22]:
# p15
@chain df_customer begin
    filter(row -> startswith(row.status_cd, ('A', 'B', 'C', 'D', 'E', 'F')), _)
    filter(row -> endswith(row.status_cd, ('1', '2', '3', '4', '5', '6', '7', '8', '9')), _)
    first(10)
end

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15
1,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
2,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
3,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
4,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
5,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
6,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
7,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
8,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
9,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1


In [23]:
# p16
# get.(match.(r"(\d+)-(\d+)-(\d+)", df_store.tel_no), 4, "No Match")

In [24]:
# p17
@chain sort!(df_customer, :birth_day) begin
    first(10)
end

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15
1,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
2,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
3,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
4,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
5,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
6,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
7,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
8,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
9,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


In [25]:
# p18
@chain df_customer begin
    sort!(:birth_day, rev = true)
    first(10)
end

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15
1,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
2,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
3,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
4,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
5,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
6,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
7,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
8,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
9,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F


In [26]:
# p19
df_receipt.ranking = competerank(df_receipt.amount, rev = true)
@chain df_receipt begin
    sort!(:amount, rev = true)
    select(:customer_id, :amount, :ranking)
    first(10)
end

Row,customer_id,amount,ranking
,String15,Int64,Int64
1,CS011415000006,10925,1
2,ZZ000000000000,6800,2
3,CS028605000002,5780,3
4,CS015515000034,5480,4
5,ZZ000000000000,5480,4
6,ZZ000000000000,5480,4
7,ZZ000000000000,5440,7
8,CS021515000089,5440,7
9,CS015515000083,5280,9


In [27]:
# p20
df_receipt.ranking_ordinal = ordinalrank(df_receipt.amount, rev = true)
@chain df_receipt begin
    sort!(:amount, rev = true)
    select(:customer_id, :amount, :ranking_ordinal)
    first(10)
end

Row,customer_id,amount,ranking_ordinal
,String15,Int64,Int64
1,CS011415000006,10925,1
2,ZZ000000000000,6800,2
3,CS028605000002,5780,3
4,CS015515000034,5480,4
5,ZZ000000000000,5480,5
6,ZZ000000000000,5480,6
7,ZZ000000000000,5440,7
8,CS021515000089,5440,8
9,CS015515000083,5280,9


In [28]:
# p21
df_receipt |> nrow

104681

In [29]:
# p22
@chain df_receipt begin
    select(:customer_id)
    unique!
    nrow
end

8307

In [30]:
# p23
@chain df_receipt begin
    groupby(:store_cd)
    combine([:amount, :quantity] .=> sum)
end

Row,store_cd,amount_sum,quantity_sum
,String7,Int64,Int64
1,S14006,712839,2284
2,S13008,809288,2491
3,S14028,786145,2458
4,S14042,534689,1935
5,S14025,755581,2394
6,S13003,764294,2197
7,S14024,736323,2417
8,S14040,701858,2233
9,S13020,796383,2383


In [31]:
# p24
@chain df_receipt begin
    groupby(:customer_id)
    combine(:sales_ymd => maximum)
    first(10)
end

Row,customer_id,sales_ymd_maximum
,String15,Int64
1,CS011415000006,20191029
2,ZZ000000000000,20191031
3,CS028605000002,20190416
4,CS015515000034,20191010
5,CS021515000089,20191027
6,CS015515000083,20190117
7,CS017414000114,20190721
8,CS028515000029,20190828
9,CS035615000040,20190830


In [32]:
# p25
@chain df_receipt begin
    groupby(:customer_id)
    combine(:sales_ymd => minimum)
    first(10)
end

Row,customer_id,sales_ymd_minimum
,String15,Int64
1,CS011415000006,20170111
2,ZZ000000000000,20170101
3,CS028605000002,20170521
4,CS015515000034,20170103
5,CS021515000089,20170117
6,CS015515000083,20170721
7,CS017414000114,20170224
8,CS028515000029,20171130
9,CS035615000040,20181102


In [33]:
# p26
@chain df_receipt begin
    groupby(:customer_id)
    combine(_, [:sales_ymd, :sales_ymd] .=> [maximum, minimum])
    filter(row -> row.sales_ymd_maximum != row.sales_ymd_minimum ,_) # _は中間データを指す．
    first(10)
end

Row,customer_id,sales_ymd_maximum,sales_ymd_minimum
,String15,Int64,Int64
1,CS011415000006,20191029,20170111
2,ZZ000000000000,20191031,20170101
3,CS028605000002,20190416,20170521
4,CS015515000034,20191010,20170103
5,CS021515000089,20191027,20170117
6,CS015515000083,20190117,20170721
7,CS017414000114,20190721,20170224
8,CS028515000029,20190828,20171130
9,CS035615000040,20190830,20181102


In [34]:
# p27
@chain df_receipt begin
    groupby(:store_cd)
    combine(:amount => mean)
    sort(:amount_mean, rev = true)
    first(5)
end

Row,store_cd,amount_mean
,String7,Float64
1,S13052,402.867
2,S13015,351.112
3,S13003,350.916
4,S14010,348.791
5,S13001,348.47


In [35]:
# p28
@chain df_receipt begin
    groupby(:store_cd)
    combine(:amount => median)
    sort(:amount_median, rev = true)
    first(5)
end

Row,store_cd,amount_median
,String7,Float64
1,S13052,190.0
2,S14010,188.0
3,S14050,185.0
4,S13003,180.0
5,S14040,180.0


In [36]:
# p29
@chain df_receipt begin
    groupby(:store_cd)
    combine(:amount => mode)
    sort(:amount_mode, rev = true)
    first(5)
end

Row,store_cd,amount_mode
,String7,Int64
1,S13003,1100
2,S13020,1100
3,S13041,1100
4,S13019,1100
5,S14040,298


In [37]:
# p30
@chain df_receipt begin
    groupby(:store_cd)
    combine(:amount => var)
    sort(:amount_var, rev = true)
    first(5)
end

Row,store_cd,amount_var
,String7,Float64
1,S13052,4.41863e5
2,S14011,3.06442e5
3,S14034,2.97068e5
4,S13001,2.95559e5
5,S13015,2.95427e5


In [38]:
# p31
@chain df_receipt begin
    groupby(:store_cd)
    combine(:amount => std)
    sort(:amount_std, rev = true)
    first(5)
end

Row,store_cd,amount_std
,String7,Float64
1,S13052,664.728
2,S14011,553.572
3,S14034,545.04
4,S13001,543.653
5,S13015,543.532


In [39]:
# p32
@chain df_receipt begin
    groupby(:store_cd)
    combine(:amount => x -> quantile(x, [0, 0.25, 0.5, 0.75, 1]))
end

Row,store_cd,amount_function
,String7,Float64
1,S14006,10.0
2,S14006,102.0
3,S14006,175.0
4,S14006,288.0
5,S14006,5060.0
6,S13008,10.0
7,S13008,102.0
8,S13008,179.0
9,S13008,295.25


In [40]:
# p33
@chain df_receipt begin
    groupby(:store_cd)
    combine(:amount => mean)
    subset(:amount_mean => x -> x .>=330)
end

@chain df_receipt begin
    groupby(:store_cd)
    combine(:amount => mean)
    filter(row -> row.amount_mean >= 330, _)
end

Row,store_cd,amount_mean
,String7,Float64
1,S13003,350.916
2,S13020,337.88
3,S14026,332.341
4,S13015,351.112
5,S14045,330.082
6,S13019,330.209
7,S14011,335.718
8,S13004,330.944
9,S14010,348.791


In [41]:
# p34
mean_value = @chain df_receipt begin
    filter(row -> !startswith(row.customer_id, "Z"), _)
    groupby(:customer_id)
    combine(:amount => sum)
    combine(:amount_sum => mean)
    _[1,1]
end

2547.742234529256

In [42]:
# p35
@chain df_receipt begin
    filter(row -> !startswith(row.customer_id, "Z"), _)
    groupby(:customer_id)
    combine(:amount => mean)
    filter(row -> row.amount_mean >= mean_value, _)
end

Row,customer_id,amount_mean
,String15,Float64
1,CS008513000063,2694.0
2,CS011315000133,2579.0
3,CS003803000016,2580.0
4,CS013413000099,2554.0
5,CS012815000021,2551.0
6,CS015615000147,2634.0
7,CS001615000201,2550.0
8,CS035412000064,2564.0


In [43]:
# p36
@chain innerjoin(df_receipt, df_store, on = :store_cd) begin
        select(names(df_receipt), :store_name)
        first(10)
    end

Row,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,ranking,ranking_ordinal,store_name
,Int64,Int64,String7,Int64,Int64,String15,String15,Int64,Int64,Int64,Int64,String31
1,20190903,1567468800,S14011,1192,2,CS011415000006,P080104055,1,10925,1,1,日吉本町店
2,20190304,1551657600,S14046,1152,2,ZZ000000000000,P090302118,1,6800,2,2,北山田店
3,20170921,1505952000,S14028,1192,2,CS028605000002,P090907210,1,5780,3,3,二ツ橋店
4,20170721,1500595200,S13015,1112,1,CS015515000034,P090907208,1,5480,4,4,南砂店
5,20190103,1546473600,S14021,1162,2,ZZ000000000000,P090907208,1,5480,4,5,伊勢原店
6,20190724,1563926400,S13009,1192,2,ZZ000000000000,P090907209,1,5480,4,6,用賀店
7,20180621,1529539200,S13004,1172,2,ZZ000000000000,P090907206,1,5440,7,7,鷺宮店
8,20190209,1549670400,S14021,1142,2,CS021515000089,P090907206,1,5440,7,8,伊勢原店
9,20181219,1545177600,S13015,1132,1,CS015515000083,P071301009,1,5280,9,9,南砂店


In [44]:
# p37
@chain innerjoin(df_product, df_category, on = [:category_medium_cd, :category_small_cd]) begin
        select(names(df_product), :category_small_name)
        first(10)
    end

Row,\ufeffproduct_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
,String15,Int64,Int64,Int64,Int64?,Int64?,String
1,P040101001,4,401,40101,198,149,弁当類
2,P040101002,4,401,40101,218,164,弁当類
3,P040101003,4,401,40101,230,173,弁当類
4,P040101004,4,401,40101,248,186,弁当類
5,P040101005,4,401,40101,268,201,弁当類
6,P040101006,4,401,40101,298,224,弁当類
7,P040101007,4,401,40101,338,254,弁当類
8,P040101008,4,401,40101,420,315,弁当類
9,P040101009,4,401,40101,498,374,弁当類


In [45]:
# p38
p38 = @chain df_receipt begin
    groupby(:customer_id)
    combine(:amount => sum)
    leftjoin(df_customer,_, on = :customer_id)
    filter(row -> row.gender_cd == 1, _)
    filter(row -> !startswith(row.customer_id, "Z"),_)
end
replace!(p38.amount_sum, missing => 0) # coalesce.(p38.amount_sum, 0) # 別解
first(p38, 10)

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,amount_sum
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15,Int64?
1,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6,812
2,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1,247
3,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1,828
4,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D,4641
5,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F,11528
6,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6,438
7,CS035112000001,宮下 ひろ子,1,女性,2007-01-11,12,152-0035,東京都目黒区自由が丘**********,S13035,20150227,7-20101003-6,1174
8,CS029113000009,長田 沙知絵,1,女性,2006-11-25,12,279-0004,千葉県浦安市猫実**********,S12029,20150903,5-20100823-5,762
9,CS023114000002,葛西 茜,1,女性,2006-10-10,12,144-0046,東京都大田区東六郷**********,S14023,20150117,A-20100228-D,2262


In [46]:
# 39
p39_sale = @chain df_receipt begin
    filter(row -> !startswith(row.customer_id, "Z"), _)
    groupby(:customer_id)
    combine(nrow)
    sort!(:nrow, rev = true)
    first(20)
end

p39_amount = @chain df_receipt begin
    filter(row -> !startswith(row.customer_id, "Z"), _)
    groupby(:customer_id)
    combine(:amount => sum)
    first(20)
end

outerjoin(p39_sale, p39_amount, on = "customer_id")

Row,customer_id,nrow,amount_sum
,String15,Int64?,Int64?
1,CS040214000008,46,missing
2,CS015415000185,44,missing
3,CS010214000010,44,missing
4,CS028415000007,42,missing
5,CS010214000002,42,missing
6,CS017415000097,40,missing
7,CS016415000141,40,missing
8,CS021514000045,40,missing
9,CS007515000107,40,missing


In [47]:
# p40
# 意図がよくわからない
nrow(df_store) * nrow(df_product)

531590

In [48]:
# p41
# using ShiftedArrays: lag
@chain df_receipt begin
    groupby(:sales_ymd)
    combine(:amount => sum)
    sort!(:sales_ymd)
    transform(_, :amount_sum => (x -> x - lag(x)) => :amout_sum_diff)
    first(10)
end

Row,sales_ymd,amount_sum,amout_sum_diff
,Int64,Int64,Int64?
1,20170101,33723,missing
2,20170102,24165,-9558
3,20170103,27503,3338
4,20170104,36165,8662
5,20170105,37830,1665
6,20170106,32387,-5443
7,20170107,23415,-8972
8,20170108,24737,1322
9,20170109,26718,1981


In [49]:
# p42
@chain df_receipt begin
    groupby(:sales_ymd)
    combine(:amount => sum)
    sort!(:sales_ymd)
    transform(_, :amount_sum => (x -> lag(x, 1)) => :amount_sum_lag_1)
    transform(_, :amount_sum => (x -> lag(x, 2)) => :amount_sum_lag_2)
    transform(_, :amount_sum => (x -> lag(x, 3)) => :amount_sum_lag_3)
    first(10)
end

Row,sales_ymd,amount_sum,amount_sum_lag_1,amount_sum_lag_2,amount_sum_lag_3
,Int64,Int64,Int64?,Int64?,Int64?
1,20170101,33723,missing,missing,missing
2,20170102,24165,33723,missing,missing
3,20170103,27503,24165,33723,missing
4,20170104,36165,27503,24165,33723
5,20170105,37830,36165,27503,24165
6,20170106,32387,37830,36165,27503
7,20170107,23415,32387,37830,36165
8,20170108,24737,23415,32387,37830
9,20170109,26718,24737,23415,32387


In [176]:
# p43
@chain leftjoin(df_receipt, df_customer, on = :customer_id) begin
    transform(:age => (x -> floor.(x / 10) * 10) => :age_bin)
    groupby([:age_bin, :gender_cd])
    combine(:amount => sum)
    unstack(:age_bin, :gender_cd, :amount_sum, allowmissing=true)
    rename!(["age_bin", "female", "male", "unknown", "missing"])
    select(:age_bin, :female, :male, :unknown)
end

Row,age_bin,female,male,unknown
,Float64?,Int64?,Int64?,Int64?
1,10.0,1591,149836,4317
2,20.0,72940,1363724,44328
3,30.0,177322,693047,50441
4,40.0,19355,9320791,483512
5,50.0,54320,6685192,342923
6,60.0,272469,987741,71418
7,70.0,13435,29764,2427
8,80.0,46360,262923,5111
9,90.0,missing,6260,missing


In [ ]:
# p44
@chain leftjoin(df_receipt, df_customer, on = :customer_id) begin
    transform(:age => (x -> floor.(x / 10) * 10) => :age_bin)
    groupby([:age_bin, :gender_cd])
    combine(:amount => sum)
    unstack(:gender_cd, :age_bin, :amount_sum, allowmissing=true)
end

Row,gender_cd,10.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0,90.0,missing
,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?
1,0,1591,72940,177322,19355,54320,272469,13435,46360,missing,missing
2,1,149836,1363724,693047,9320791,6685192,987741,29764,262923,6260,missing
3,9,4317,44328,50441,483512,342923,71418,2427,5111,missing,missing
4,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,12395003


In [ ]:
# p45
# using Dates
@chain df_customer begin
    transform(:birth_day => ( x -> Dates.format.(x, "yyyymmdd")) => :birth_day_str) # broadcastでformatを適用する．
    select(:customer_id, :birth_day_str)
    first(10)
end
#Dates.format.(df_customer.birth_day, "yyyymmdd")


Row,customer_id,birth_day_str
,String15,String
1,CS035114000004,20071125
2,CS022103000002,20071002
3,CS002113000009,20070917
4,CS004115000014,20070809
5,CS002114000010,20070603
6,CS025115000002,20070418
7,CS002113000025,20070330
8,CS033112000003,20070322
9,CS007115000006,20070310


In [103]:
# p46
@chain df_customer begin
    transform(:application_date => (x -> Date.(string.(x), DateFormat("yyyymmdd"))) => :application_date_date)
    select(:customer_id, :application_date_date)
    first(10)
end

Row,customer_id,application_date_date
,String15,Date
1,CS035114000004,2015-06-19
2,CS022103000002,2016-09-09
3,CS002113000009,2016-03-04
4,CS004115000014,2016-11-20
5,CS002114000010,2016-09-20
6,CS025115000002,2016-01-16
7,CS002113000025,2017-10-30
8,CS033112000003,2015-06-06
9,CS007115000006,2015-11-18


In [106]:
# p47
@chain df_receipt begin
    transform(:sales_ymd => (x -> Date.(string.(x), DateFormat("yyyymmdd"))) => :sales_ymd_date)
    select(:receipt_no, :receipt_sub_no, :sales_ymd_date)
    first(10)
end

Row,receipt_no,receipt_sub_no,sales_ymd_date
,Int64,Int64,Date
1,1192,2,2019-09-03
2,1152,2,2019-03-04
3,1192,2,2017-09-21
4,1112,1,2017-07-21
5,1162,2,2019-01-03
6,1192,2,2019-07-24
7,1172,2,2018-06-21
8,1142,2,2019-02-09
9,1132,1,2018-12-19


In [134]:
# p48
@chain df_receipt begin
    transform(:sales_epoch => (x -> unix2datetime.(x)) => :sales_epoch_datetime)
    transform(:sales_epoch_datetime => (x -> Date.(x)) => :sales_epoch_date)
    select(:receipt_no, :receipt_sub_no, :sales_epoch_date)
    first(10)
end
#unix2datetime.(df_receipt.sales_epoch)

Row,receipt_no,receipt_sub_no,sales_epoch_date
,Int64,Int64,Date
1,1192,2,2019-09-03
2,1152,2,2019-03-04
3,1192,2,2017-09-21
4,1112,1,2017-07-21
5,1162,2,2019-01-03
6,1192,2,2019-07-24
7,1172,2,2018-06-21
8,1142,2,2019-02-09
9,1132,1,2018-12-19


In [139]:
# p49
@chain df_receipt begin
    transform(:sales_epoch => (x -> unix2datetime.(x)) => :sales_epoch_datetime)
    transform(:sales_epoch_datetime => (x -> Dates.year.(x)) => :sales_epoch_year)
    select(:receipt_no, :receipt_sub_no, :sales_epoch_year)
    first(10)
end

Row,receipt_no,receipt_sub_no,sales_epoch_year
,Int64,Int64,Int64
1,1192,2,2019
2,1152,2,2019
3,1192,2,2017
4,1112,1,2017
5,1162,2,2019
6,1192,2,2019
7,1172,2,2018
8,1142,2,2019
9,1132,1,2018


In [184]:
# p50
@chain df_receipt begin
    transform(:sales_epoch => (x -> unix2datetime.(x)) => :sales_epoch_datetime)
    transform(:sales_epoch_datetime => (x -> Dates.month.(x)) => :sales_epoch_month)
    transform(:sales_epoch_month => (x -> string.(x, pad = 2)) => :sales_epoch_month_str) # pad = 2で2桁にする．
    select(:receipt_no, :receipt_sub_no, :sales_epoch_month_str)
    first(10)
end

Row,receipt_no,receipt_sub_no,sales_epoch_month_str
,Int64,Int64,String
1,1192,2,09
2,1152,2,03
3,1192,2,09
4,1112,1,07
5,1162,2,01
6,1192,2,07
7,1172,2,06
8,1142,2,02
9,1132,1,12


In [185]:
# p51
@chain df_receipt begin
    transform(:sales_epoch => (x -> unix2datetime.(x)) => :sales_epoch_datetime)
    transform(:sales_epoch_datetime => (x -> Dates.day.(x)) => :sales_epoch_day)
    transform(:sales_epoch_day => (x -> string.(x, pad = 2)) => :sales_epoch_day_str)
    select(:receipt_no, :receipt_sub_no, :sales_epoch_day_str)
    first(10)
end

Row,receipt_no,receipt_sub_no,sales_epoch_day_str
,Int64,Int64,String
1,1192,2,03
2,1152,2,04
3,1192,2,21
4,1112,1,21
5,1162,2,03
6,1192,2,24
7,1172,2,21
8,1142,2,09
9,1132,1,19


In [257]:
# p52
@chain df_receipt begin
    filter(row -> !startswith(row.customer_id, "Z"),_)
    groupby(:customer_id)
    combine(:amount => sum)
    transform(:amount_sum => (x -> Int.(x .> 2000)) => :amount_sum_over_2000)
    first(10)
end

Row,customer_id,amount_sum,amount_sum_over_2000
,String15,Int64,Int64
1,CS011415000006,16094,1
2,CS028605000002,11270,1
3,CS015515000034,15300,1
4,CS021515000089,17580,1
5,CS015515000083,9229,1
6,CS017414000114,8745,1
7,CS028515000029,8536,1
8,CS035615000040,5608,1
9,CS030515000189,11913,1


In [270]:
# p53
@chain df_customer begin
    transform(:postal_cd => (x -> parse.(Int, SubString.(x, 1, 3))) => :postal_cd_prefix)
    transform(:postal_cd_prefix => (x -> Int.(100 .<= x .<= 209)) => :postal_cd_dummy)
    leftjoin(df_receipt, _, on =:customer_id)
    unique!(:customer_id)
    groupby(:postal_cd_dummy)
    combine(nrow)
end

Row,postal_cd_dummy,nrow
,Int64?,Int64
1,0,3906
2,1,4400
3,missing,1


In [305]:
# p54
@chain df_customer begin
    transform(:address => (x -> Int.(occursin.("東京都", x)) .* 13) => :address_tokyo)
    transform(:address => (x -> Int.(occursin.("神奈川県", x)) .* 14) => :address_kanagawa)
    transform(:address => (x -> Int.(occursin.("埼玉県", x)) .* 11) => :address_saitama)
    transform(:address => (x -> Int.(occursin.("千葉県", x)) .* 12) => :address_chiba)
    transform(_, [:address_tokyo, :address_kanagawa, :address_saitama, :address_chiba] => (+) => :address_code)

    select(:customer_id, :address, :address_code)
    first(10)
end

Row,customer_id,address,address_code
,String15,String,Int64
1,CS035114000004,東京都世田谷区桜**********,13
2,CS022103000002,神奈川県逗子市逗子**********,14
3,CS002113000009,東京都小金井市貫井南町**********,13
4,CS004115000014,東京都中野区上鷺宮**********,13
5,CS002114000010,東京都小金井市貫井北町**********,13
6,CS025115000002,神奈川県横浜市泉区上飯田町**********,14
7,CS002113000025,東京都小金井市貫井北町**********,13
8,CS033112000003,神奈川県横浜市戸塚区名瀬町**********,14
9,CS007115000006,千葉県佐倉市西志津**********,12


In [ ]:
# p55
quantiles = @chain df_receipt begin
    groupby(:customer_id)
    combine(:amount => sum)
    combine(:amount_sum => x -> quantile(x, [0.25, 0.5, 0.75]))
    _.amount_sum_function
end
q1 = quantiles[1]; q2 = quantiles[2]; q3 = quantiles[3]

@chain df_receipt begin
    groupby(:customer_id)
    combine(:amount => sum)
    transform(:amount_sum => (x -> ifelse(x .< q1, 1, 0) )
    #combine(:amount_sum => x -> quantile(x, [0.25, 0.5, 0.75]))
    #_.amount_sum_function
end

Row,customer_id,amount_sum
,String15,Int64
1,CS011415000006,16094
2,ZZ000000000000,12395003
3,CS028605000002,11270
4,CS015515000034,15300
5,CS021515000089,17580
6,CS015515000083,9229
7,CS017414000114,8745
8,CS028515000029,8536
9,CS035615000040,5608
